# <font color='red'>DO NOT MODIFY THIS FILE! MAKE A COPY</font>

## Template for loading trARPES data from the Phoibos detector at FHI Berlin.
The data are loaded as xarray following a conversion to the [NeXus format](https://manual.nexusformat.org/classes/contributed_definitions/NXmpes.html#nxmpes) using the [Nomad Parser Nexus](https://github.com/nomad-coe/nomad-parser-nexus).

In [ ]:
# Provide the scan to load
scan = 12000

First, the SpecsScan class is imported which has the scan loader as its class method.

In [ ]:
%load_ext autoreload
%autoreload 2
from specsscan import SpecsScan
import matplotlib.pyplot as plt

Here, a SpecsScan class instance is created as per the configuration provided in [config.yaml](../tests/data/config.yaml). The user may set the entries in config.yaml file, for example, the data path and conversion parameters as per the requirements before creating this instance.

The path may changed to point to the scan folder of the data of interest (for example, on a server drive)

In [ ]:
metadata = {}
# Here, manual metadata can be added. Alternatively, they can come from an Electronic Lab Notebook below.
# Uncomment for adding manual Metadata (delete """-lines)
"""
#General
#metadata['experiment_summary'] = 'Summar'
metadata['entry_title'] = 'Title'
#metadata['experiment_title'] = 'Exp Title'

#User
# Fill general parameters of NXuser
#metadata['user0'] = {}
#metadata['user0']['name'] = 'Laurenz Rettig'
#metadata['user0']['role'] = 'Principal Investigator'
#metadata['user0']['affiliation'] = 'Fritz Haber Institute of the Max Planck Society'
#metadata['user0']['address'] = 'Faradayweg 4-6, 14195 Berlin'
#metadata['user0']['email'] = 'rettig@fhi-berlin.mpg.de'

metadata['instrument'] = {}
# energy resolution
metadata['instrument']['energy_resolution'] = 150.
metadata['instrument']['electronanalyser'] = {}
metadata['instrument']['electronanalyser']['energy_resolution'] = 120
metadata['instrument']['electronanalyser']['angular_resolution'] = 0.2
metadata['instrument']['electronanalyser']['spatial_resolution'] = 0.5

#probe beam
metadata['instrument']['beam']={}
metadata['instrument']['beam']['probe']={}
metadata['instrument']['beam']['probe']['incident_energy'] = 21.7
metadata['instrument']['beam']['probe']['incident_energy_spread'] = 0.11
metadata['instrument']['beam']['probe']['pulse_duration'] = 20.
metadata['instrument']['beam']['probe']['frequency'] = 500.
metadata['instrument']['beam']['probe']['incident_polarization'] = [1, 1, 0, 0] # p pol Stokes vector
metadata['instrument']['beam']['probe']['extent'] = [80., 80.]
#pump beam
#metadata['instrument']['beam']['pump']={}
#metadata['instrument']['beam']['pump']['incident_energy'] = 1.55
#metadata['instrument']['beam']['pump']['incident_energy_spread'] = 0.08
#metadata['instrument']['beam']['pump']['pulse_duration'] = 35.
#metadata['instrument']['beam']['pump']['frequency'] = 500.
#metadata['instrument']['beam']['pump']['incident_polarization'] = [1, -1, 0, 0] # s pol Stokes vector
#metadata['instrument']['beam']['pump']['incident_wavelength'] = 800.
#metadata['instrument']['beam']['pump']['average_power'] = 224.
#metadata['instrument']['beam']['pump']['pulse_energy'] = metadata['instrument']['beam']['pump']['average_power']/metadata['instrument']['beam']['pump']['frequency']#µJ
#metadata['instrument']['beam']['pump']['extent'] = [300/4*2.34, 270/4*2.35] #Gaussian 4sigma -> FWHM
#metadata['instrument']['beam']['pump']['fluence'] = 1.00
#metadata['instrument']['beam']['pump']['delay'] = 0.02

#sample
metadata['sample']={}
#metadata['sample']['preparation_date'] = '2017-03-19T10:00:00+00:00'
#metadata['sample']['preparation_description'] = 'Cleaved'
#metadata['sample']['sample_history'] = 'Cleaved in UHV'
#metadata['sample']['chemical_formula'] = 'TbTe3'
#metadata['sample']['description'] = 'cleaved single crystal of TbTe3'
metadata['sample']['name'] = 'TbTe3 Single Crystal'
""" and None

In [ ]:
sps = SpecsScan(system_config="../config.yaml")

### Check scan for sample degradation
Enable the following cell to inspect the scan

In [ ]:
"""
res_check = sps.check_scan(
    scan=scan,
    delays=0, # for a fixed delay of index 0
)
# Some plots
fig, axs = plt.subplots(1, 3, figsize=(8, 2.5), constrained_layout=True)
res_check.loc[{'Angle':slice(-5, 5)}].sum(axis=(0)).plot(ax=axs[0])
res_check[:,:,0].T.plot(ax=axs[1])
res_check[:,:,-1].T.plot(ax=axs[2])
""" and None

### Convert scan
The load_scan method loads the scan as an xarray along with the metadata needed for nexus conversion. The progress bars can be activated by changing the config parameter, enable_nested_progress_bar, to true in config.yaml 

In [ ]:
res = sps.load_scan(
    scan=scan,
    metadata=metadata,
    collect_metadata=True,
    # iterations=[0, 3, 6],
)

### Save scan to NeXus

In [ ]:
sps.save(f"Scan{scan}.nxs")

### Plots

In [ ]:
# Some plots
fig, axs = plt.subplots(3, 1, figsize=(6, 10), constrained_layout=True)
res.sum(axis=(2)).T.plot(ax=axs[0])
res.sum(axis=(0)).T.plot(ax=axs[1])
res.sum(axis=(1)).T.plot(ax=axs[2])
plt.show()